In [ ]:
from google.colab import drive
import zipfile
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pathlib
import glob as gb
import glob
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

drive.mount('/content/drive')

zip_file_path = "/content/drive/MyDrive/archive (2) (1).zip"
extract_dir = "/content"

if os.path.exists(zip_file_path):
  try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
      zip_ref.extractall(extract_dir)
    print(f"Successfully extracted '{zip_file_path}' to '{extract_dir}'")
  except zipfile.BadZipFile:
    print(f"Error: '{zip_file_path}' is not a valid zip file.")
  except Exception as e:
    print(f"An error occurres: {e}")
else:
  print(f"Error: '{zip_file_path}' does not exist.")

train = '/content/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/train'
size = 224
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range = 90,
    width_shift_range = 0.0,
    height_shift_range = 0.0,
    shear_range = 0.0,
    zoom_range = 0.0,
    horizontal_flip = False,
    vertical_flip = False,
    rescale = 1/255.0,
    preprocessing_function = None,
    validation_split = 0.1,
).flow_from_directory(train,
                      batch_size = 164,
                      target_size = (size,size),
                      subset = "training",
                      color_mode = 'rgb',
                      class_mode = 'categorical',
                      shuffle = True)

valid = '/content/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/valid'
valid_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1/255.0,
    preprocessing_function = None,
    validation_split = 0.1,
).flow_from_directory(valid,
                      batch_size = 164,
                      target_size = (224,224),
                      subset = 'validation',
                      color_mode = 'rgb',
                      class_mode = 'categorical',
                      shuffle = False)

test = '/content/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/valid'
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1/255.0,
    preprocessing_function = None,
).flow_from_directory(test,
                      batch_size = 164,
                      target_size = (224,224),
                      color_mode = 'rgb',
                      class_mode = 'categorical',
                      shuffle = False)



base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(38, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

early_stopping = EarlyStopping(monitor = 'val_loss', patience = 15, restore_best_weights = True)
model_checkpoint = ModelCheckpoint('best_model.keras', monitor = 'val_loss', save_best_only = True)
model_ReduceLROnPlateau = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patience = 15, min_lr = 0.000001)
callbacks = [early_stopping, model_checkpoint, model_ReduceLROnPlateau]

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=["accuracy", "precision", "recall"])

history = model.fit(
    train_generator,
    epochs=15,
    validation_data=valid_generator,
    callbacks=callbacks
)

model.save('MobileNetV2_plantdiseases_model.keras')

loss, accuracy, precision, recall = model.evaluate(test_generator)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Successfully extracted '/content/drive/MyDrive/archive (2) (1).zip' to '/content'
Found 63282 images belonging to 38 classes.
Found 1742 images belonging to 38 classes.
Found 17572 images belonging to 38 classes.
Epoch 1/15
386/386 ━━━━━━━━━━━━━━━━━━━━ 789s 2s/step - accuracy: 0.7898 - loss: 0.7351 - precision: 0.9053 - recall: 0.7082 - val_accuracy: 0.8961 - val_loss: 0.3072 - val_precision: 0.9252 - val_recall: 0.8737 - learning_rate: 0.0010
Epoch 2/15
386/386 ━━━━━━━━━━━━━━━━━━━━ 796s 2s/step - accuracy: 0.9300 - loss: 0.2099 - precision: 0.9453 - recall: 0.9162 - val_accuracy: 0.9076 - val_loss: 0.2696 - val_precision: 0.9305 - val_recall: 0.8915 - learning_rate: 0.0010
Epoch 3/15
386/386 ━━━━━━━━━━━━━━━━━━━━ 802s 2s/step - accuracy: 0.9432 - loss: 0.1653 - precision: 0.9542 - recall: 0.9335 - val_accuracy: 0.9173 - val_loss: 0.2461 - val_precision: 0.929